# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
        print("到達頁面底端")
        html_source = browser.page_source
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)
        else:
            if len(p) > 1:
                for child in p.children:
                    if ((child.string) is not None):
                        print(child.string)
                    else:
                        continue

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 慧洋海運去年每股賺3.41元　換約租金漲幅將提升至12%營運看好

▲針對船隊規劃，慧洋海運今年預計將新進7艘節能船，其中2艘將配置脫硫器，以增添船隊優勢及調度的彈性。(圖／慧洋海運提供)
記者林淑慧／台北報導
企業自結2019年財報陸續出爐。散裝航運業者慧洋今（6）日公佈2019全年自結合併營收為138.33億元，稅前淨利達22.16億元，分別較前年同期增加5.86%與18.78%，自結每股稅前盈餘（EPS）則為3.41元，獲利表現優於2018年的3.02元水準。
散裝航運產業去（2019）年因巴西淡水河谷礦災、美中貿易談判拉鋸遞延等諸多外在因素，影響運價議價空間，慧洋於2019年的平均租金換約漲幅為4%，2019年12月合併營收為新台幣11.24億元，自結單月稅前淨利為新台幣1.19億元，每股稅前盈餘則為0.18元。
▲看好節能新造船帶動船隊升級，慧洋海運營運長趙子隆(圖左)與財務長薛亦駿，樂觀看待今年集團營運更上一層樓。（圖／慧洋提供）
慧洋表示，今年預計有40艘船舶進行換約，由於美中談判將於本月15日簽署第一階段協議，全球穀物運輸需求可望回補，再加上環保法規的實施可望調節市場供需，公司預期換約漲幅將可望提升至12%，可望一舉擺脫過去的低潮，迎接春燕到來。
在船隊規劃方面，慧洋2019全年總計交付4艘節能新船並汰換7艘舊船，今年則預估將新進7艘節能船，其中2艘將配置脫硫器，以增添船隊優勢及調度的彈性，並依市場價格適時處分5至10艘老舊船舶，維持慧洋船隊運力，目標規模將落在125至130艘且平均船齡6年左右的最佳狀態。
面對今年元旦上路的限硫令，慧洋指出，由於中小型船舶加裝脫硫器之成本較高，回本年限長，因此散裝航運普遍共識以使用低硫油為主。至於目前市場上高、低硫油報價差距擴大、供給問題浮現的情況下，由於慧洋租約船比重較高，燃油成本為租家負擔，因此油價差距對公司營運衝擊較小，加上慧洋旗下擁有四成以上的節能船，相較於其他同業更具競爭優勢，整體效益貢獻將在今年明顯擴大。
----------------------------------------------------------------------

[1]

▲康友的印尼疫苗廠。（圖／康友提供）
記者姚惠茹／台北報導
康友-KY（6452）今（6）日宣布繼取得二張禽流感藥證幫印尼政府代工後，再度取得重大進程，與亞洲最大畜牧集團泰國正大集團合作，由印尼正大集團旗下PT SHS INTERNACIONAL獨家代理帝斯Harbindo品牌疫苗，未來正大集團SHS將獨家銷售帝斯禽流感疫苗及後續取得其他藥證的疫苗。
康友表示，正大集團在印尼養雞版圖高達40％市占率，年約16億隻，並在台灣市佔年約6億隻，自育種、孵化、飼養、屠宰到銷售一條龍生產管理，年產值超過1,000億台幣，若含其它畜牧及養殖，營收絕對超過1,500億台幣。
康友指出，正大集團擁有印尼最龐大市占率，最先進檢測設備及人員，就只缺疫苗廠，歷經幾個月的不斷檢測，帝斯禽流感疫苗療效深獲肯定，不僅讓正大集團自身採用，也要求為其獨家總代理，並銷售給其他60％業者，未來也有機會行銷到中國大陸之外的市場。
康友分享，帝斯為東南亞設備最先進及最大廠，為求得雙贏，歷經考量而給予Harbindo品牌獨家總代理權，而正大集團取得總代理權，價格也並未下降，對於剩下60％市場，也因正大集團登高一呼，有機會加速推廣。
康友強調，印尼禽類疫苗市場，隨著肉雞每年20％成長而成長，一年約有2億美元，複合成長率約20％，光2023年就有4億美元，而印尼政府預計2~4年間逐步關閉禽類疫苗進口，複合成長率不只20％，並隨著後面三張二價疫苗及一張三價疫苗取得後，可望挹注康友營運更上一層樓。
----------------------------------------------------------------------

[12] 台指期大跌137點　分析師：震盪往下機率大、稍安勿躁

▲台指期1月6日走勢。（圖／取自期交所）
記者余弦妙／台北報導
台指期今（6）日開盤下跌70點，來到12,017點，但避險賣壓湧現，行情開低走低，一度來到11,948點，終場以11,950點作收，下跌137點，與現貨相較，逆價差3.36點。展望後市，統一期貨投顧部經理廖恩平分析，近期的策略就是建議投資人稍安勿躁，觀察一下賣壓情況，再決定是否要進場，且明日盤勢應該還是會呈現震盪往下機率較大。
廖恩平表示，今天的整體國際股市皆受到中東地緣政治的影響，所以台股近期走勢會偏弱，再加上前一波漲勢偏高，所以在年前，內外資會有獲


包括中華電信在內的5大電信商6日仍展現強烈企圖加價爭取5G入場門票，使得標金仍持續上升，金額突破1000億元之後，下一步將挑戰2013年4G頻譜競標，當時6家業者歷經40天393回合競價後，以總標金1186.5億元爭取270MHz的紀錄。




5G 6日競標情況
1800MHz
3500MHz
28000MHz
由NCC主辦的5G競價，自12月10日開始，每日上午9時至下午4時進行10回合，由中華電信、台灣大哥大、台灣之星、亞太電信及遠傳電信等5家現有行動業者參與。



NCC表示，第一階段數量競價改良前年4G第三波釋照所採用的「同時、多回合及上升競價法(SMRA)」，當對可用區塊的需求大於或等於提供數量時，則該頻段的回合價格將在下一回合中提高1.03倍，並持續至連續二回合所有有權提出需求頻寬之競價者，都沒有對任何一個頻段提出有效需求頻寬時即告結束。最新競價結果及詳細資料請參考NCC網站(https://mbb5g.ncc.gov.tw/)。
----------------------------------------------------------------------

[23] 重新定義智慧車用人機介面！　友達CES 2020秀「數位駕駛艙解決方案」

▲友達超大型異型切割曲面駕駛艙顯示器。（圖／友達提供）
記者姚惠茹／台北報導
友達（2409）今（6）日宣布，將在1月7日至10日參加2020年美國消費性電子展（CES 2020），為使用者在居家、工作和行進間，打造未來智慧聯網車的最新顯示技術和應用，展出亮點包括實現駕駛艙全面數位化的關鍵顯示技術。

 

智慧家居、電競和個人穿戴裝置等高階顯示應用方面，友達將展示包括家庭娛樂的8K4K超高解析技術、採用Mini LED背光的高階HDR電競顯示技術，以及用於智慧手錶的先進AMOLED技術，推出全球最窄2毫米（mm）錶殼邊框、具備可置放真正錶針的中央通孔、超低功耗和低刷新率，以及內嵌NFC模組可支援各式系統交易數據傳輸的真圓形AMOLED面板。
▲友達9.6吋異型切割LTPS後視鏡顯示器。（圖／友達提供）

 

尖端車用顯示技術方面，由5G和AIoT形塑的新智慧場域，促使汽車成為僅次於家庭和辦公室的第三重要使用空間，數位駕駛艙因而成為智慧車用技術發展重心，而友達異型切割曲面駕駛艙顯示器運用高解

▲保力達B飲品的瓶蓋，多年前遭環保署驗出含PVC。（圖／取自保力達公司官方網頁）
記者吳銘峯／台北報導
「明天的氣力，今天尬你傳便便」、「福氣啦、福氣啦」以多支電視廣告，深植人心的「保力達B」，驚傳使用違規物質。環保署驗出，保力達B螺旋瓶在2008年到2012年間使用的瓶蓋內墊，含有致癌疑慮的物質PVC，因此依照《廢棄物清理法》追徵3億餘元。保力達公司不滿，打起行政訴訟敗訴確定後，聲請釋憲。司法院大法官6日舉行說明會，聽取意見，而保力達公司認為相關規定不夠明確，才會遭罰，請求宣告相關規定違憲。至於相對機關環保署，則主張廢棄物清理收費制度行之有年，且頗獲好評，並無違憲問題。
PVC聚氯乙烯（Poly vinyl chloride）屬於塑膠的一種，是由氯乙烯單體（Vinyl chloride monomer, 簡稱VCM）聚合而成的塑膠，在臺灣的塑膠分類編號是三號。由於VCM在聚合過程中若不完全，將產生致癌物質，影響人體，國際癌症研究署IARC在1987年將VCM氯乙烯列為確定對人類有致癌性的物質（第一級致癌物），並將PVC聚氯乙烯列為尚不確定致癌性的物質（第三級致癌物）。目前國際間也逐漸減少使用PVC產品，儘管廠商不斷宣稱PVC無毒無害，但網友間仍以「毒塑膠」稱呼PVC。
而環保署在2012年底將保力達B玻璃容器螺旋鋁蓋內墊材質送SGS檢驗，以紅外光譜分析及焰色法檢測結果均含有PVC，經詢問保力達公司後，確認保力達公司在2008年到2012年間生產的保力達B，均使用相同瓶蓋，因此環保署追徵3億零8百餘萬元的廢棄物清理費用。保力達公司不滿，提出行政訴訟敗訴確定後，聲請大法官解釋。大法官認為相關問題太過專業，因此藉憲法法庭召開說明會，聽取意見。
保力達公司主張，本案屬行政機關課徵的「特別公課」，並非依法制定、收取的「稅賦」，故《廢棄物清理法》的法律保留密度不足，也違反授權明確性原則，甚至以「空白授權」的方式向人民收費，違反憲法保障的法律保留原則。另外一小片PVC瓶蓋的處理費用，甚至高過整個PVC空瓶處理費用，也違反平等原則，進而侵害人民的財產權，請求大法官宣告相關規定違憲。
相對機關環保署則表示，《廢棄物清理法》的公告收費流程、標準，乃考量國際間「延伸生產者責任制度」，讓生產者所負擔的責任延伸到消費完的物品上，才能確保環境永續生存。而公告收費標準乃經過立法院三讀通過的法律

新聞節目中心／綜合報導
去年台股大好，勞退基金績效很好，開春新聞就說截至2019年11月底前，獲利2,312.4億元，收益率9.96%，所有新制1,177萬名勞工有望在農曆春節後領到紅包，每名勞工平均可分配到1.96萬元；以近一年有提繳的688萬餘戶有效帳戶計算，每個帳戶平均可分配到3.36萬元。大家覺得光靠政府的退休金真的可以過活嗎?你覺得退休一個月要多少錢才夠花? 既然勞退基金大賺錢，我們跟著勞退買股票，樂當存股族。
你覺得40歲開始存退休金會不會太晚? 退休教授告訴你一點都不晚，他45歲開始靠存股，3年快速累積千萬資產。今天三位達人要告訴你，可以讓你抱一輩子的好股票，讓你輕鬆鬆存退休金。不過一開春台股就連漲，再次站穩12000大關，像現在股市這麼熱，股價那麼高，存股族是不是不適合進場? 那該何時進場呢? 縱橫台股美股技術大師要告訴你何時才是進場甜蜜點，存股達人要教你何時要將股票脫手的好時機，這些精彩節目內容不要錯過，保證讓你一開春2020賺飽飽。
▼《ETtoday新聞雲》全新製作財經節目—《雲端最有錢》每周一下午1點10分至2點直播。（圖／新聞節目中心）。
▼加入臉書粉絲頁訂閱直播通知也可以看得到喔！
 
 
關於《雲端最有錢》
老闆總是不加薪，想投資又不知道該怎麼規劃? 上沖下洗的股市如何套利? 區塊鏈加密貨幣又是什麼? 2019景氣混沌不明，資產怎麼配置才能錢滾錢? 從台股震盪中賺到買菜金、從基金佈局中賺到旅遊金、從房產配置中越換越大間!《ETtoday新聞雲》推出全新節目《雲端最有錢》由資深財經媒體人邱沁宜擔任主持人，帶您輕鬆聊天讀懂財經，最賺錢的經驗分享、最新潮的投資趨勢、虧大錢的地雷閃避，都在《雲端最有錢》與您分享!
資深財經媒體人邱沁宜，20年投資資歷，專訪過逾百位上市櫃公司CEO，台灣唯一擁有理財規劃師證照的財經主播，著有「投資越簡單越好賺」、「理財好像談戀愛」等書，台大北大EMBA畢業，永遠在探索世界的學習者，相信人生最棒的投資就是投資自己。在《雲端最有錢》要帶你探索世界新趨勢，搞懂生活中的投資密碼，ㄧ起邁向財富自由人生自由之路。
《ETtoday新聞雲》新聞直播中心全新製作財經節目—《雲端最有錢》於2019年1月16日起，每周一下午1點10分至2點，將在《ETtoday新聞雲》的官網、新聞雲、筋斗雲、播吧等臉書粉絲頁、東森新聞雲APP線上同

▲根據金管會統計，2019年前11月行動支付總交易金額達1,003億元，較去年同期大幅成長139%。（示意圖／達志影像）
記者紀佳妘／台北報導
金管會最新公布金融機構辦理行動支付情形，2019年前11月行動支付交易金額達1,003億元，較2018年同期大幅成長139%，意味著行動支付市場進入快速成長期。
金管會2016年來推動各項行動支付服務，以滿足民眾對金融服務的需求。行動支付工具共有6項，包括手機綁信用卡（TSM）、手機綁電子票證、行動金融卡、掃碼支付（QR code）、電子支付機構O2O及行動收單機（mPOS），其中最多人使用的就是手機綁定信用卡付款。
根據銀行局統計，2016年行動支付總交易額為14.9億元，2017年交易額大幅成長至148億元，2018年則暴增為478億元，到了2019年前11月更是衝破千億元大關，來到1,003億元，較去年同期的418億元，大幅成長139%，顯見近年來行動支付交易額都有跳躍式成長。
金管會主委顧立雄曾表示，行動支付市場進入快速成長期，預估2019全年行動支付總交易額可望突破千億元。如今，從數據中來看，單是2019年前11月就來到1,003億元，率先突破千億元。
----------------------------------------------------------------------

[53] 一審遭判賠惠普133億懲罰性賠償　廣明：會上訴維護權益

▲廣明光電遭控違反美國反托拉斯法，需判賠惠普4.39億美元，將提出上訴，維護公司與股東權益。（示意圖／pakutaso）
財經中心／綜合報導
廣達旗下廣明光電遭控違反美國反托拉斯法，2019年10月遭判賠惠普1.76億美元（約新台幣53億元），不過美國法院最新裁定，賠償金額要提高至4.39億美元（約新台幣133億元）。
廣明光電今天發布重訊指出，本公司及子公司Quanta Storage America, Inc.連同等其他被告接獲南德州聯邦地方法院就光碟機限制價格反托拉斯訴訟案的判決，應賠給惠普（HP）4.39億美元。
依據聯邦法，就陪審團作出事實損害賠償金額認定後，向法院申請3倍懲罰性賠償，進而由地方法院法官行使裁量權。不過在宣判前，美國司法部及台灣公平會均在事實上認定本廣明光電及子公司Quanta Storage America, Inc.並無違反美國

▲隨著農曆春節即將到來，民營金控獲利表現亮眼，預估年終獎金可超越上年。（示意圖／記者周宸亘攝）
記者紀佳妘／台北報導
農曆新年即將到來，銀行業年終獎金行情也陸續揭曉，多家民營金控表態預期年終獎金可望超越上年，預計春節前發放，同時也有為員工加薪的機會，最高調薪幅度達10%。
根據金管會統計，金融業2019年前11月稅前盈餘達6,211.41億元，較去年同期成長17.9%，一舉創下歷史新高水準，其中全體銀行業稅前盈餘達3,851.6億元，年增7.2%、保險業稅前盈餘也達1,839億元，年增49.3%。
各家金融業者年終獎金並未完全出爐，不過2019年獲利表現亮眼，可望反映在年終獎金上。富邦金表示，每年都會視年度獲利狀況於年前發放年終獎金，平均約4至6個月，但仍視各子公司業務獲利及個人績效表現而定。
此外，為符合市場薪資競爭力，富邦每年都會依員工績效表現，定期辦理調薪，近年每年都有3%至5%調薪率，預期2020年也應該有此水準。
國泰金也將視公司營運績效、盈餘、員工績效表現核發年終獎金，同時也會參考市場調薪率及公司整體薪資競爭力後，針對晉升與員工績效表現，定期進行薪資檢視及差異化調整，以達激勵與留置優秀人才效果。
台新金指出，2019年公司整體績效表現佳，預計年獎平均將比2018年水準提高1個月，並依員工績效再差異化核給；績效佳者最高可超過12個月，本年度將待結算後規劃發放；中信金也將比照往年，依獲利水準提撥年終獎金，預計將於農曆年前發放。
玉山金表示，除了每年9月發給同仁1個月的員工酬勞外，年終獎金平均4至6個月。玉山獲利持續成長，年終獎金預估也較往年略增，會在春節前夕進行年終獎金發放。
此外，玉山日前也宣布，從今年1月1日起全面調薪，平均調薪幅度超過8%，其中第一波調薪中，主要聚焦基層同仁及中階主管的薪資結構，提高固定薪的金額，基層同仁每個月本薪提高3,000元，中階主管提高主管加給，全體同仁平均提升4.6%，基層同仁平均提升6.4%。
在4月1日後，加計第1季經理人、中階主管及同仁的晉升及績效的第二波調薪後，全體同仁平均薪資將提升超過8%，基層同仁平均薪資將提升達10%以上。
----------------------------------------------------------------------

[61] 選舉投票、春節返鄉潮！防非洲豬瘟　海關

▲廣達31周年尾牙，董事長林百里。（圖／記者周康玉攝）
記者姚惠茹／台北報導
正值尾牙旺季，每年又以科技廠尾牙最有看頭，今年電子代工五哥鴻海、仁寶、和碩、廣達、緯創之中，廣達上個月18日的尾牙就送出達4000萬元的總獎金，最大單一獎金更是高達30萬元現金，直接嗨翻現場員工，接下來鴻海、仁寶、和碩、緯創尾牙也都將陸續登場。
今年農曆春節較往年早，因此陸續已經有不少企業舉辦尾牙，其中廣達已經在上個月18日舉行《齊心聚力創精彩，邁向AI大未來》尾牙活動，在南港展覽館席開830桌，參與員工人數超過8000人，當天就送出達4000萬元的總獎金，最大單一獎金更是高達30萬元現金，並一口氣送出10位頭獎。
仁寶今年尾牙預計在1月16日登場，和碩則在1月17日舉辦尾牙，相較上個年度送出破億元的總獎金，兩大廠發錢不手軟的程度，今年獎金總額有得拚場，同樣讓員工十分期待，還有緯創的獎金規模預計也有千萬以上的規模。
鴻海愛心嘉年華今年預計在1月22日登場，今年總計有約3.5萬名員工共襄盛舉，而且活動當天也會有員工眷屬獎的抽獎活動，以去年為例，10萬元的眷屬獎一路加碼，最後發出350個獎項，總共發出3500萬元。
鴻海今年依然邀請到張菲擔任主持，藝人表演名單陣容也十分堅強，包括十鼓擊樂團、Maniac Family、HRC CONCRETE KiDS、彭佳慧、李炳輝、聲林之王（Gail、李艾薇、劉學甫、陳忻玥、吳霏、艾文、林怡廷、屁孩）、信（蘇見信）、Boxing（葛西瓦、洛克斯、阿六、曉龍、曉明、好樂迪）、持修、安那、張惠妹共53名藝人輪番演唱。
----------------------------------------------------------------------

[71] 勞保局e化服務上線！　投保可直接線上預辦加保或退保

▲勞保局e化服務上線！（圖／記者李毓康攝）
記者余弦妙／台北報導
勞保局為讓投保單位更加便利投保或退保，自今年1月起，建立「e化服務系統」只要確認員工預定到、離職日前10日內預辦員工加保或退保，讓申報作業更便利。
勞保局表示，由於投保單位反映，早已確定員工到職日期，但因到職當日疏漏申報加保，導致影響員工權益，希望勞保局能夠提供更方便作業方式，因此回應投保單位需求，勞保局108年就已完成，針對已申辦「e化服務系統」之單位提供勞工保險得線上預辦加

▲東森購物2019年全通路整體業績表現亮眼，會員及業績均大有成長，東森購物董事長王令麟表示，其中「東森幣」的帶動功不可沒。（圖／東森購物提供）
記者洪菱鞠／台北報導
東森（2614）旗下轉投資東森購物，2019年1～12月合併草莓網營收年成長22%，達204億元，衝破200億大關。全通路整體業績表現亮眼，主要歸功於整合全集團資源力推東森幣奏效，推出至今已發行超過54億東森幣，成為目前所有零售、電商通路中發行量最大的「幣王」，成效斐然也令同業紛起效尤。
為全面提升東森購物會員服務，東森幣自2018年8月開始發送後，累積至2019年12月31日為止，17個月共發行超過54億，其中有28億已被使用，使用率逾53%，不僅是目前國內電商幣中發行量最高，也是虛擬通路最早發行的貨幣。而目前可透過4種方式獲得，包括：會員生日禮金領取、購物回饋、加入東森直銷網連通會員，立即可得500元東森幣、下載東森購物APP玩奪寶遊戲賺東森幣。
▲東森幣取得方式容易，可透過生日會員禮、簽到打卡、購物回饋等可獲得，尤其即日起至1月31日以前，在東森購物各通路消費，最高可享20%東森幣回饋。（圖／東森購物提供）
由於東森幣1元等同現金1元，可於消費時折抵使用，也能在東森購物APP上玩遊戲和抽獎，在多用途助攻下，帶動東森購物2019年會員及業績均有成長，而根據數據統計分析，全通路會員成長新客及舊客甦醒達9.3%，其中VIP成長佔30%；回購率從過去使用比率8.9%提高為10.8%；發行前購買件數由1078萬件躍升至發行後1353萬件，增加26%。
東森購物王令麟董事長表示，東森幣推出迄今，東森購物的訂單數成長19%、件數成長26%、營業額成長33%、折抵金額成長56%。2019年11月VIP會員人數為43.4萬，與去年2018年同期33.4萬相比，大幅成長30%。東森購物VIP人數累計至2019年11月份總計為43.3萬人，共創造125.8億消費金額，佔全通路營收73.5%，無論人數與營收都創下5年以來的新高，東森幣居功厥偉。展望未來，集團期在2024年整體營收目標達1300億。
東森購物20年來始終不變的是東森嚴選，一直在變的是不斷追求創新。東森購物目前除了有電視、型錄、電話行銷、MOD、APP、OTT等，集團近年更積極發展多元通路，包括併購自然美、草莓網、寵物雲，帶動東森聲量往上提升之外，各通路之間

▲
精測總經理黃水可。（圖／資料照）
記者周康玉／台北報導
半導體測試介面廠精測（6510）今（3）日公布去（2019）年12月份營收達3.1億元，月減9.8%，年增43.8%；去（2019）年第4季單季營收達10.1億元，季減8.6%，年增39.8 %；全年總營收為33.9億元，較前一年度成長3.3%，優於預期。
受惠於去美化效應，精測第3季獲利大跳升，單季每股賺7.55元，累計前三季每股獲利達14.01元，雖較去年同期16.82元為低，然而法人預估，在去美化持續發酵下，精測第4季業績較第3季小幅下滑在10%以內，預估單季業績有機會逼近新台幣10億元，單季可望創下同期新高。



精測表示，第4季為產業傳統淡季，公司持續優化產品組合，擴大新市場布局。此外，全新垂直式探針卡（VPC；Vertical Probe Card）產品線也於該季度開始擴充產能，並逐步貢獻營運成績，VPC去年營收比重達雙位數，符合預期。



展望未來，5G、AI將引領半導體產業發展趨勢，中華精測將順勢發展，積極招募電機、電子、電控、光學、化工、材料、機械及資訊等相關科系之研發人才，為5G+AI之應用發展，厚植VPC智慧製造之實力，以提供客戶更完善的All In House服務。
精測日前宣布，為了優化產能效率，終止兩年前與美商的特定用途PCB生產計劃，認列損失不超過1億元。法人認為，精測決定斬斷非核心事業，將在第4季認列非現金資產損失，但也因此在今（2020）、明（2021）兩年折舊費用減輕，預估第4季獲利下修38.6%，同時調升今年全年獲利增加0.9%。
法人表示，根據通路調查，將持續追蹤一款5奈米5G智慧手機AP產品案於去（2019）年12月初始進入量產，第1季產量逐月增長，4月全速放量。同時，亦有顯卡廠首度導入探針卡PCB，對精測在先進測試市場維持主導地位有信心，營運展望無虞。
法人看好精測受惠5G、7奈米及更先進製程成長動能，預估今、明兩年全年每股盈餘可突破30元，相當於三個股本；但需注意5G行動通訊的需求可能受到貿易戰影響，而由盛轉衰；以及探針卡市場競爭加劇等風險。
----------------------------------------------------------------------

[91] 新台幣仍守30元關卡　收盤升值0.9分收30.105元兌1美元
